# template から HTML と JS を生成する
## テスト用のデータを test-tsumugi にコピー


In [1]:
import os
import json
import gzip
import shutil
from pathlib import Path
from collections import defaultdict

import pandas as pd

In [2]:
# Move up to top directory

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/TSUMUGI-dev-main/notebooks/notebools-web
/mnt/e/Research/TSUMUGI-dev-main


# 古いファイルの消去


In [3]:
%%bash

rm -rf test-tsumugi/data
rm -rf test-tsumugi/app


In [4]:
%%bash

mkdir -p test-tsumugi/data/phenotype
mkdir -p test-tsumugi/data/genesymbol
mkdir -p test-tsumugi/app/phenotype
mkdir -p test-tsumugi/app/genesymbol
mkdir -p test-tsumugi/app/genelist/

cp -r TSUMUGI/css test-tsumugi/ # copy top's CSS
cp -r TSUMUGI/js test-tsumugi/ # copy top's JS

cp -r TSUMUGI/template/css test-tsumugi/app/ # copy app's CSS
cp -r TSUMUGI/template/js test-tsumugi/app/ # copy app's JS


## index.html の描画に必要なファイルを生成する


In [5]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    if not Path("data", "network", "mp_term_name", f"{mp_term}.json.gz").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    mp_terms[mp_term_name_space] = mp_term

In [6]:
print(list(mp_terms)[:3])
print(len(mp_terms))  # 515

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
525


In [7]:
json.dump(mp_terms, open("data/overlap/available_mp_terms.json", "w"), indent=2)
pd.DataFrame(mp_terms.keys()).to_csv("data/overlap/available_mp_terms.txt", index=False, header=False)

In [8]:
%%bash

cp data/overlap/available_mp_terms.json test-tsumugi/data/available_mp_terms.json
cp data/overlap/available_mp_terms.txt test-tsumugi/data/available_mp_terms.txt
cp data/overlap/available_gene_symbols.txt test-tsumugi/data/available_gene_symbols.txt
cp data/annotation/symbol_mgiid.json test-tsumugi/data/marker_symbol_accession_id.json


## index.html を生成


In [9]:
%%bash

grep -v "REMOVE_THIS_LINE" TSUMUGI/template/template_index.html > test-tsumugi/index.html

# appのHTMLとJSを生成


### binariyの表現型を抽出

In [10]:
path_files = Path("TSUMUGI", "data", "phenotype").glob("*.json.gz")
binary_phenotypes = []
for path_file in path_files:
    mp_term_name = path_file.name.replace(".json.gz", " ")
    with gzip.open(path_file, "rt") as f:
        json_data: list[dict] = json.load(f)

    count_node_color = defaultdict(int)
    for data in json_data:
        if "node_color" not in data["data"]:
            continue
        count_node_color[data["data"]["node_color"]] += 1
    if len(count_node_color) == 1 and 1 in count_node_color:
        binary_phenotypes.append(mp_term_name)

print(len(binary_phenotypes))
print(binary_phenotypes[:3])
Path("data", "annotation", "binary_phenotypes.txt").write_text("\n".join(binary_phenotypes))

154
['abnormal_abdominal_wall_morphology ', 'abnormal_adrenal_gland_morphology ', 'abnormal_allantois_morphology ']


3761

## Phenotype

In [11]:
# ========== ファイル処理ユーティリティ ==========


def read_file(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        return f.read()


def write_file(filepath, content):
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)


# ========== データ収集系関数 ==========


def get_target_phenotypes(dir_path, target_phenotypes: list[str] | None = None):
    targets = []
    for file in Path(dir_path).glob("*.csv"):
        name = file.stem
        if target_phenotypes is None:
            targets.append(name)
        else:
            if name.startswith(target_phenotypes):
                targets.append(name)
    return targets


def get_impc_url(mp_term_name_space, tsv_path):
    with open(tsv_path, encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if parts[0] == mp_term_name_space:
                return parts[1]
    return ""


def is_binary(mp_term, binary_file_path):
    with open(binary_file_path) as f:
        return any(line.strip() == mp_term for line in f)


# ========== テンプレート系関数 ==========


def replace_placeholder(template, placeholder, insert_text):
    return template.replace(placeholder, insert_text)


def inject_html(template_path, insert_path, placeholder, output_path):
    template = read_file(template_path)
    insert = read_file(insert_path)
    updated = replace_placeholder(template, placeholder, insert)
    write_file(output_path, updated)


def generate_simple_html(template_path, output_path, replacements):
    content = read_file(template_path)
    for key, value in replacements.items():
        content = content.replace(key, value)
    write_file(output_path, content)


# ========== HTML生成系関数 ==========


def generate_all_html(mp_term, mp_term_name_space, impc_url, mode):
    # control-panel と cy-container
    for part in ["control-panel", "cy-container"]:
        template_path = f"TSUMUGI/template/template-app-html/{part}.html"
        output_path = f"/tmp/{part}.html"
        if mode == "non-binary-phenotype":
            insert_path = f"TSUMUGI/template/template-app-html/{part}-phenotype.html"
            inject_html(template_path, insert_path, "XXX_PHENOTYPE", output_path)
        else:
            generate_simple_html(template_path, output_path, {"XXX_PHENOTYPE": ""})

    # head.html
    generate_simple_html(
        "TSUMUGI/template/template-app-html/head.html",
        "/tmp/head.html",
        {"XXX_TITLE": mp_term_name_space, "XXX_JS_FILE_NAME": mp_term},
    )

    # header.html
    header_insert = f"<a href='{impc_url}' target='_blank'>{mp_term_name_space}</a>"
    generate_simple_html(
        "TSUMUGI/template/template-app-html/header.html", "/tmp/header.html", {"XXX_TITLE": header_insert}
    )

    # template_app.html → 完成版HTML
    template = read_file("TSUMUGI/template/template-app-html/template_app.html")
    final_html = (
        template.replace("XXX_HEAD", read_file("/tmp/head.html"))
        .replace("XXX_H1", read_file("/tmp/header.html"))
        .replace("XXX_CONTROL_PANEL", read_file("/tmp/control-panel.html"))
        .replace("XXX_CY_CONTAINER", read_file("/tmp/cy-container.html"))
    )

    write_file(f"test-tsumugi/app/phenotype/{mp_term}.html", final_html)


# ========== JavaScript生成関数 ==========


def generate_javascript(mp_term, mp_term_name_space, mode):
    template_app_path = "/tmp/template_app.js"

    if mode == "non-binary-phenotype":
        shutil.copy(
            "TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_phenotype.js",
            "/tmp/filterByNodeColorAndEdgeSize_phenotype.js",
        )

        template = read_file("TSUMUGI/template/template-app-js/template_app.js")
        node_min_max = read_file("TSUMUGI/template/template-app-js/nodeMinMax.js")
        init = read_file("TSUMUGI/template/template-app-js/node_color_initialization.js")
        update = read_file("TSUMUGI/template/template-app-js/node_color_update.js")
        template = (
            template.replace("XXX_NODE_MIN_MAX", node_min_max)
            .replace("XXX_NODE_COLOR_INITIALIZATION", init)
            .replace("XXX_NODE_COLOR_UPDATE", update)
        )
        write_file(template_app_path, template)

    else:
        # Binary phenotype の処理
        lines = read_file("TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_phenotype.js").splitlines()
        filtered_lines = "\n".join(line for line in lines if "REMOVE_THIS_LINE_IF_BINARY_PHENOTYPE" not in line)
        write_file("/tmp/filterByNodeColorAndEdgeSize_phenotype.js", filtered_lines)

        template = read_file("TSUMUGI/template/template-app-js/template_app.js")
        template = template.replace("XXX_NODE_COLOR_INITIALIZATION", "").replace("XXX_NODE_COLOR_UPDATE", "")
        write_file(template_app_path, template)

    # 最終JS生成
    main_template = read_file(template_app_path)
    insert = read_file("/tmp/filterByNodeColorAndEdgeSize_phenotype.js")

    final_js = (
        main_template.replace("XXX_NODE_MIN_MAX", "")
        .replace("XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE", insert)
        .replace("XXX_EDGE_MAX", "const edgeMax = Math.max(...edgeSizes);")
        .replace("XXX_ELEMENTS", f"loadJSONGz('../../data/phenotype/{mp_term}.json.gz')")
        .replace("XXX_NAME", mp_term)
    )

    write_file(f"test-tsumugi/app/phenotype/{mp_term}.js", final_js)


In [12]:
# ========== メイン処理 ==========
target_phenotypes = ("male_infertility", "increased_fasting", "preweaning_lethality")
phenotypes = get_target_phenotypes("data/mp_term_name", target_phenotypes)

for mp_term in phenotypes:
    print(f"Processing: {mp_term}")
    mp_term_name_space = mp_term.replace("_", " ")
    impc_url = get_impc_url(mp_term_name_space, "data/annotation/mptermname_phenotypeurl.tsv")
    mode = "binary_phenotype" if is_binary(mp_term, "data/annotation/binary_phenotypes.txt") else "non-binary-phenotype"

    # データコピー
    shutil.copy(f"data/network/mp_term_name/{mp_term}.json.gz", f"test-tsumugi/data/phenotype/{mp_term}.json.gz")

    generate_all_html(mp_term, mp_term_name_space, impc_url, mode)
    generate_javascript(mp_term, mp_term_name_space, mode)


Processing: increased_fasting_circulating_glucose_level
Processing: male_infertility
Processing: preweaning_lethality,_complete_penetrance
Processing: preweaning_lethality,_incomplete_penetrance


## Gene Symbol

In [13]:
# ========== Gene Symbol用 HTML生成関数 ==========
def generate_gene_html(gene_symbol, impc_url):
    # control-panel と cy-container
    for part in ["control-panel", "cy-container"]:
        template_path = f"TSUMUGI/template/template-app-html/{part}.html"
        output_path = f"/tmp/{part}.html"
        generate_simple_html(template_path, output_path, {"XXX_PHENOTYPE": ""})

    # head.html
    generate_simple_html(
        "TSUMUGI/template/template-app-html/head.html",
        "/tmp/head.html",
        {"XXX_TITLE": gene_symbol, "XXX_JS_FILE_NAME": gene_symbol},
    )

    # header.html
    header_insert = f"<a href='{impc_url}' target='_blank'>{gene_symbol}</a>"
    generate_simple_html(
        "TSUMUGI/template/template-app-html/header.html", "/tmp/header.html", {"XXX_TITLE": header_insert}
    )

    # template_app.html
    template = read_file("TSUMUGI/template/template-app-html/template_app.html")
    final_html = (
        template.replace("XXX_HEAD", read_file("/tmp/head.html"))
        .replace("XXX_H1", read_file("/tmp/header.html"))
        .replace("XXX_CONTROL_PANEL", read_file("/tmp/control-panel.html"))
        .replace("XXX_CY_CONTAINER", read_file("/tmp/cy-container.html"))
    )

    write_file(f"test-tsumugi/app/genesymbol/{gene_symbol}.html", final_html)


# ========== JavaScript生成関数 ==========
def generate_gene_javascript(gene_symbol):
    # template_app.js のベース作成（特定部分除去）
    template_lines = read_file("TSUMUGI/template/template-app-js/template_app.js").splitlines()
    filtered_lines = [
        line
        for line in template_lines
        if "XXX_NODE_COLOR_INITIALIZATION" not in line and "XXX_NODE_COLOR_UPDATE" not in line
    ]
    write_file("/tmp/template_app.js", "\n".join(filtered_lines))

    # JS中に埋め込むコードを差し込み
    template = read_file("/tmp/template_app.js")
    insert_filterByNodeColorAndEdgeSize = read_file(
        "TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_genesymbol.js"
    )
    insert_edgeMax = read_file("TSUMUGI/template/template-app-js/edgeMax_on_genesymbol.js")

    final_js = (
        template.replace("XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE", insert_filterByNodeColorAndEdgeSize)
        .replace("XXX_NODE_MIN_MAX", "")
        .replace("XXX_EDGE_MAX", insert_edgeMax)
        .replace("XXX_ELEMENTS", "loadJSONGz('../../data/genesymbol/XXX_NAME.json.gz')")
        .replace("XXX_NAME", gene_symbol)
    )

    write_file(f"test-tsumugi/app/genesymbol/{gene_symbol}.js", final_js)


# ========== MGI ID 取得関数 ==========
def get_mgi_id(gene_symbol, tsv_path="data/annotation/symbol_mgiid.tsv"):
    with open(tsv_path, encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if parts[0] == gene_symbol:
                return parts[1]
    return ""

In [14]:
# ========== メイン処理 ==========
target_genes = ["Rab10", "Asxl1", "Ddx46", "Ap3b2", "Kcnma1"]
with open("data/overlap/available_gene_symbols.txt", encoding="utf-8") as f:
    available = {line.strip() for line in f}

for gene_symbol in target_genes:
    if gene_symbol not in available:
        continue

    print(f"Processing: {gene_symbol}")

    # データコピー
    src = f"data/network/gene_symbol/{gene_symbol}.json.gz"
    dst = f"test-tsumugi/data/genesymbol/{gene_symbol}.json.gz"
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(src, dst)

    # URL生成
    mgi_id = get_mgi_id(gene_symbol)
    impc_url = f"https://www.mousephenotype.org/data/genes/{mgi_id}"

    # HTMLとJS生成
    generate_gene_html(gene_symbol, impc_url)
    generate_gene_javascript(gene_symbol)


Processing: Rab10
Processing: Asxl1
Processing: Ddx46
Processing: Ap3b2
Processing: Kcnma1


## Gene List

In [15]:
# ========== Gene List用 HTML生成関数 ==========
def generate_genelist_html():
    # control-panel と cy-container
    for part in ["control-panel", "cy-container"]:
        template_path = f"TSUMUGI/template/template-app-html/{part}.html"
        output_path = f"/tmp/{part}.html"
        generate_simple_html(template_path, output_path, {"XXX_PHENOTYPE": ""})

    # head.html
    generate_simple_html(
        "TSUMUGI/template/template-app-html/head.html",
        "/tmp/head.html",
        {"XXX_TITLE": "Gene List", "XXX_JS_FILE_NAME": "network_genelist"},
    )

    # header.html
    header_insert = "gene list"
    generate_simple_html(
        "TSUMUGI/template/template-app-html/header.html", "/tmp/header.html", {"XXX_TITLE": header_insert}
    )

    # template_app.html
    template = read_file("TSUMUGI/template/template-app-html/template_app.html")
    final_html = (
        template.replace("XXX_HEAD", read_file("/tmp/head.html"))
        .replace("XXX_H1", read_file("/tmp/header.html"))
        .replace("XXX_CONTROL_PANEL", read_file("/tmp/control-panel.html"))
        .replace("XXX_CY_CONTAINER", read_file("/tmp/cy-container.html"))
    )

    write_file("test-tsumugi/app/genelist/network_genelist.html", final_html)


# ========== JavaScript生成関数 ==========
def generate_genelist_javascript():
    # template_app.js のベース作成（特定部分除去）
    template_lines = read_file("TSUMUGI/template/template-app-js/template_app.js").splitlines()
    filtered_lines = [
        line
        for line in template_lines
        if "XXX_NODE_COLOR_INITIALIZATION" not in line and "XXX_NODE_COLOR_UPDATE" not in line
    ]
    write_file("/tmp/template_app.js", "\n".join(filtered_lines))

    # JS中に埋め込むコードを差し込み
    template = read_file("/tmp/template_app.js")
    insert = read_file("TSUMUGI/template/template-app-js/filterByNodeColorAndEdgeSize_genelist.js")

    final_js = (
        template.replace("XXX_FILTER_BY_NODE_COLOR_AND_EDGE_SIZE", insert)
        .replace("XXX_NODE_MIN_MAX", "")
        .replace("XXX_EDGE_MAX", "const edgeMax = Math.max(...edgeSizes);")
        .replace("XXX_ELEMENTS", "JSON.parse(localStorage.getItem('elements'))")
        .replace("XXX_NAME", "geneList")
    )

    write_file("test-tsumugi/app/genelist/network_genelist.js", final_js)


In [16]:
# ========== メイン処理 ==========
# HTMLとJS生成
generate_genelist_html()
generate_genelist_javascript()


In [17]:
%%bash

prettier --write \
    "test-tsumugi/index.html" \
    "test-tsumugi/js/**/*" \
    "test-tsumugi/app/**/*" \
    --print-width 120 --prose-wrap never --tab-width 4 > /dev/null 2>&1



In [18]:
%%bash

uname -a  # OS name
date +"%Y/%m/%d %H:%M:%S"  # Last update

Linux Sycom-2021 5.15.167.4-microsoft-standard-WSL2 #1 SMP Tue Nov 5 00:21:55 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
2025/04/19 11:51:59
